<a href="https://colab.research.google.com/github/Masum06/TinyAgent/blob/main/TinyAgent_Colab_Noebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

In [2]:
_set_env("OPENAI_API_KEY")

OPENAI_API_KEY: ··········


In [3]:
!pip install -q tiktoken

# TinyAgent

In [4]:
import json
import tiktoken
from openai import OpenAI

client = OpenAI()

In [12]:
# @title
class TinyAgent:

    def __init__(self, model, tokenizer=None, debug=False):

        self.model = model
        self.messages = []
        self.max_tokens = 3072
        self.debug = False
        self.reasoning_effort = "low"
        self.temperature = 1

    def clear_messages(self):
        self.messages = list()

    def add_message(self,message_type, message):
        self.messages.append({"role": message_type, "content":message})

    def add_system_message(self, message):
        self.add_message("system", message)

    def add_user_message(self, message):
        self.add_message("user", message)

    def set_max_tokens(self, max_tokens):
        self.max_tokens = max_tokens

    def set_debug(self, debug):
        self.debug = debug

    def set_reasoning_effort(self, reasoning_effort):
        self.reasoning_effort = reasoning_effort

    def call(self, prompt="", response_type="text"):
        if prompt:
            self.add_user_message(prompt)

        if "gpt-5" in self.model:
            response = client.responses.create(
                model=self.model,
                input=self.messages,
                reasoning={"effort": self.reasoning_effort},
                text={
                    "format": {
                      "type": response_type
                    },
                    "verbosity": "low"
                  },
            )
            reply = response.output_text
            if self.debug:
                print(reply)
            return reply

        elif "gpt-4" in self.model:
            response = client.responses.create(
              model=self.model,
              input=self.messages,
              temperature=self.temperature,
              max_output_tokens=self.max_tokens,
              top_p=1,
              text={
                "format": {
                  "type": response_type # "text", "json_object"
                }
              }
            )
            reply = response.output_text
            self.add_message("assistant", reply)
            if self.debug:
                print(reply)
            return reply

    def load_json(self,s):
        import json
        try:
            return json.loads(s)
        except json.JSONDecodeError:
            return None

    def call_reply(self):

        reply = self.call()

        return reply

    def call_json(self, prompt=""):
        self.add_system_message("Reply must be JSON format.")
        if prompt:
            self.add_user_message(prompt)
        reply = self.call(response_type="json_object")
        if not reply:
            print("Empty reply")
            return None

        reply = reply.strip()
        if reply.startswith("```json"):
            reply = reply[len("```json"):].strip()
            if reply.endswith("```"):
                reply = reply[:-3].strip()

        # Use OR, and guard length
        if not (reply.startswith("{") and reply.endswith("}")):
            print("Not JSON structure")
            return None

        try:
            return self.load_json(reply)
        except Exception:
            print("Error parsing JSON")
            return None

In [13]:
tiny_agent = TinyAgent("gpt-4.1")

In [14]:
tiny_agent.messages

[]

## Unstructured Call

In [9]:
tiny_agent.call("What's your name?")

'I’m ChatGPT—an AI language model created by OpenAI. I don’t have a personal name, but you can just call me ChatGPT! How can I help you today?'

In [10]:
tiny_agent.messages

[{'role': 'user', 'content': "What's your name?"},
 {'role': 'assistant',
  'content': 'I’m ChatGPT—an AI language model created by OpenAI. I don’t have a personal name, but you can just call me ChatGPT! How can I help you today?'}]

## Structured Call

In [15]:
result = tiny_agent.call_json("Give me a list of popular math equations in JSON format.")
result

{'math_equations': [{'name': 'Pythagorean Theorem',
   'equation': 'a^2 + b^2 = c^2'},
  {'name': 'Quadratic Formula',
   'equation': 'x = \\frac{-b \\pm \\sqrt{b^2-4ac}}{2a}'},
  {'name': "Euler's Formula", 'equation': 'e^{ix} = \\cos{x} + i\\sin{x}'},
  {'name': "Einstein's Mass-Energy Equivalence", 'equation': 'E = mc^2'},
  {'name': 'Area of a Circle', 'equation': 'A = \\pi r^2'},
  {'name': 'Slope of a Line', 'equation': 'm = \\frac{y_2 - y_1}{x_2 - x_1}'},
  {'name': "Newton's Second Law", 'equation': 'F = ma'},
  {'name': 'Logarithm Definition', 'equation': 'y = \\log_b x \\iff b^y = x'},
  {'name': 'Binomial Theorem',
   'equation': '(a + b)^n = \\sum_{k=0}^{n} {n \\choose k} a^{n-k} b^k'},
  {'name': 'Derivative Definition',
   'equation': "f'(x) = \\lim_{h \\to 0} \\frac{f(x+h) - f(x)}{h}"}]}

In [17]:
result['math_equations'][0]

{'name': 'Pythagorean Theorem', 'equation': 'a^2 + b^2 = c^2'}